# 非稳态通常除霜周期时间计算

In [1]:
import pandas as pd
from pandas import DataFrame,Series
import numpy as np

In [2]:
filename='data/mine/2017年07月24日 10時14分22秒.csv'

## 输入 MAX 频率
MAX频率

除霜频率

In [3]:
max_hz=99
defrost_hz=99

## csv文件转码为utf-8

In [4]:
import csv
csvfile=open(filename,encoding='CP932')
newname=filename.replace('秒','秒_new')
newfile=open(newname,'w+',newline='\n',encoding='UTF-8')
reader=csv.reader(csvfile)
writer=csv.writer(newfile)

for line in reader:
    writer.writerow(line)

csvfile.close()
newfile.close()

In [5]:
data=pd.read_csv(newname)

## 插入时间序列

In [6]:
seconds=np.arange(0,5*data.shape[0],5)
minutes=seconds/60

In [7]:
data.insert(0,'seconds',seconds)
data.insert(0,'minutes',minutes)

### 目标频率中间跳 0 的处理
在剔除目标频率为0的数据前插入时间序列。

剔除目标频率为0的数据后，重新索引。

避免中间中目标频率异常跳0引起的index间断，避免中间跳0的时间段被忽略。

In [8]:
comp_on=data[data['COMP  運転周波数(目標)']>0]
comp_on.index=np.arange(0,comp_on.shape[0],1)

## 异常值处理

### 异常值显示

In [9]:
outlier_mask=data['COMP  運転周波数(目標)']>max_hz
if outlier_mask.any():         #存在异常值
    outlier_index=data[outlier_mask].index
    print(data.loc[outlier_index,'COMP  運転周波数(目標)'])

### 异常值修正

In [10]:
if outlier_mask.any():         #存在异常值
    outlier=data['COMP  運転周波数(目標)'][outlier_index].values
    outlier_fix=(data['COMP  運転周波数(目標)'][outlier_index-1].values+data['COMP  運転周波数(目標)'][outlier_index+1].values)/2
    data.set_value(outlier_index,'COMP  運転周波数(目標)',outlier_fix)
#     print('存在异常值，位置:',outlier_index, outlier)
#     print('异常值已修正为:',outlier_fix) 

### 修正后的值

In [11]:
if outlier_mask.any():
    print(data.loc[outlier_index,'COMP  運転周波数(目標)'])

## 获取起止时间点 `index`

In [12]:
hz_mask=comp_on['COMP  運転周波数(目標)']>(max_hz-1)   #考虑存在保护的情况

In [13]:
start_max=[]
end_max=[]
for i in hz_mask.index[:-1]:
    if not hz_mask[i] and hz_mask[i+1]:
        start_max.append(i+1)
    elif hz_mask[i] and not hz_mask[i+1]:
        end_max.append(i)

In [14]:
# columns=[['第一周期','第一周期','第二周期','第二周期','第三周期','第三周期','第四周期','第四周期','第五周期','第五周期'],
#         ['不动作','除霜','不动作','除霜','不动作','除霜','不动作','除霜','不动作','除霜']]
# index=['2017/7/20']

In [15]:
start_max,end_max

([52, 564, 596, 1048, 1081, 1521, 1554, 1993, 2026, 2466, 2498],
 [547, 580, 1031, 1065, 1504, 1538, 1976, 2010, 2449, 2482])

## 计算周期长度

In [16]:
phase_non_def={}
i=1
for m in np.arange(0,9,2):
    if m==0:
        phase_non_def[i]=comp_on['minutes'][end_max[m]] - comp_on['minutes'][0]
    else:
        phase_non_def[i]=comp_on['minutes'][end_max[m]] -comp_on['minutes'][start_max[m]] 
    i+=1

In [17]:
phase_def={}
i=1
for m in np.arange(1,10,2):
    phase_def[i]=comp_on['seconds'][end_max[m]]-comp_on['seconds'][start_max[m]]
    i+=1

In [30]:
format=lambda x: '%.2f' %x

In [56]:
df=DataFrame({'phase_def':phase_def,'phase_non_def':phase_non_def},columns=['phase_non_def','phase_def'])
df['phase_non_def']=df['phase_non_def'].apply(format)
# df.unstack().swaplevel(1,0).sort_index(level=0)
# df.T.to_csv('data/mine/defrost_out.csv')
df.T

,1,2,3,4,5
phase_non_def,45.58,36.25,35.25,35.17,35.25
phase_def,80,85,85,85,80


### Bugs

1. 中间跳出小于 `max_hz` 的频率；
2. 除霜频率 不等于 最大频率；